In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    
    secret = numpy.array([1, -2, -3.5, 6.7]) if communicator.rank == 1 else None
    log.info(f"Player {communicator.rank} secret: {secret}", src=1)    
    
    secret_share = protocol.share(src=1, secret=protocol.encoder.encode(secret), shape=(4,))
    absolute_share = protocol.absolute(secret_share)
    absolute = protocol.encoder.decode(protocol.reveal(absolute_share))
    
    log.info(f"Player {communicator.rank} abs(secret): {absolute}")

main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:50171 from tcp://127.0.0.1:50171.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:50171 from tcp://127.0.0.1:50172.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:50171 from tcp://127.0.0.1:50173.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 1 secret: [ 1.  -2.  -3.5  6.7]
INFO:root:Player 0 abs(secret): [1.         2.         3.5        6.69999695]
INFO:root:Player 1 abs(secret): [1.         2.         3.5        6.69999695]
INFO:root:Player 2 abs(secret): [1.         2.         3.5        6.69999695]
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 2

Note that $6.69999695$ is an approximation to the original value, due to the limited precision of the fixed point encoding.